<a href="https://colab.research.google.com/github/DENGXUELIN/clash/blob/main/Distillation_Toy_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [59]:
# Imports
import tensorflow as tf

from tensorflow.keras import models
from tensorflow.keras import layers

tf.random.set_seed(666)

In [60]:
# Load the FashionMNIST dataset, scale the pixel values
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
X_train = X_train/255.
X_test = X_test/255.

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((60000, 28, 28), (10000, 28, 28), (60000,), (10000,))

In [61]:
# Change the pixel values to float32 and reshape input data
X_train = X_train.astype("float32").reshape(-1, 28, 28, 1)
X_test = X_test.astype("float32").reshape(-1, 28, 28, 1)

In [62]:
# Define utility function for building a basic shallow Convnet
def get_teacher_model():
    model = models.Sequential()
    model.add(layers.Conv2D(16, (5, 5), activation="relu",
        input_shape=(28, 28, 1)))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(layers.Conv2D(32, (5, 5), activation="relu"))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(layers.Dropout(0.2))
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation="relu"))
    model.add(layers.Dense(10))

    return model

In [63]:
# Define loss function and optimizer
loss_func = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()

In [64]:
# Prepare TF dataset
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(100).batch(64)
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(64)

# Train the teacher model
teacher_model = get_teacher_model()
teacher_model.compile(loss=loss_func, optimizer=optimizer, metrics=["accuracy"])
teacher_model.fit(train_ds,
                  validation_data=test_ds,
                  epochs=10)

Epoch 1/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 10s 7ms/step - accuracy: 0.7067 - loss: 0.8034 - val_accuracy: 0.8488 - val_loss: 0.4182
Epoch 2/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.8581 - loss: 0.3952 - val_accuracy: 0.8662 - val_loss: 0.3672
Epoch 3/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.8750 - loss: 0.3400 - val_accuracy: 0.8714 - val_loss: 0.3432
Epoch 4/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.8872 - loss: 0.3061 - val_accuracy: 0.8820 - val_loss: 0.3246
Epoch 5/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.8939 - loss: 0.2851 - val_accuracy: 0.8842 - val_loss: 0.3094
Epoch 6/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.8998 - loss: 0.2683 - val_accuracy: 0.8951 - val_loss: 0.2872
Epoch 7/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9044 - loss: 0.2532 - val_accuracy: 0.8884 - val_loss: 0.2987
Epoch 8/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.9085 - loss: 0.2406 - val_accuracy: 0

In [66]:
# Evaluate and serialize
print("Test accuracy: {:.2f}".format(teacher_model.evaluate(test_ds)[1]*100))
teacher_model.save_weights("teacher_model.weights.h5")

157/157 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8965 - loss: 0.2827
Test accuracy: 90.05


In [67]:
# Student model utility
def get_student_model():
    model = models.Sequential()
    model.add(layers.Input(shape=(28, 28, 1)))
    model.add(layers.Flatten())
    model.add(layers.Dense(48, activation="relu"))
    model.add(layers.Dense(10))

    return model

In [68]:
# Credits: https://github.com/google-research/simclr/blob/master/colabs/distillation_self_training.ipynb
def get_kd_loss(student_logits, teacher_logits, temperature=0.5):
    teacher_probs = tf.nn.softmax(teacher_logits / temperature)
    kd_loss = tf.compat.v1.losses.softmax_cross_entropy(
        teacher_probs, student_logits / temperature, temperature**2)
    return kd_loss

In [69]:
# Model, optimizer
student_model = get_student_model()
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

# Average the loss across the batch size within an epoch
train_loss = tf.keras.metrics.Mean(name="train_loss")
valid_loss = tf.keras.metrics.Mean(name="test_loss")

# Specify the performance metric
train_acc = tf.keras.metrics.SparseCategoricalAccuracy(name="train_acc")
valid_acc = tf.keras.metrics.SparseCategoricalAccuracy(name="valid_acc")

In [70]:
# Train utils
@tf.function
def model_train(images, labels, teacher_model,
                student_model, optimizer, temperature):
    teacher_logits = teacher_model(images)

    with tf.GradientTape() as tape:
        student_logits = student_model(images)
        loss = get_kd_loss(student_logits, teacher_logits, temperature)

    gradients = tape.gradient(loss, student_model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, student_model.trainable_variables))

    train_loss(loss)
    train_acc(labels, tf.nn.softmax(student_logits))

In [71]:
# Validation utils
@tf.function
def model_validate(images, labels, teacher_model,
                   student_model, temperature):
    teacher_logits = teacher_model(images)

    student_logits = student_model(images)
    loss = get_kd_loss(student_logits, teacher_logits, temperature)

    valid_loss(loss)
    valid_acc(labels, tf.nn.softmax(student_logits))

In [74]:
# Tie everything together
def train_model(epochs, teacher_model, student_model, optimizer, temperature=0.5):
    for epoch in range(epochs):
        for (images, labels) in train_ds:
            model_train(images, labels, teacher_model, student_model, optimizer, temperature)

        for (images, labels) in test_ds:
            model_validate(images, labels, teacher_model, student_model, temperature)

        (loss, acc) = train_loss.result(), train_acc.result()
        (val_loss, val_acc) = valid_loss.result(), valid_acc.result()

        train_loss.reset_state(), train_acc.reset_state()
        valid_loss.reset_state(), valid_acc.reset_state()

        template = "Epoch {}, loss: {:.3f}, acc: {:.3f}, val_loss: {:.3f}, val_acc: {:.3f}"
        print (template.format(epoch+1,
                            loss,
                            acc,
                            val_loss,
                            val_acc))


    return teacher_model, student_model

In [75]:
_, student_model = train_model(10, teacher_model, student_model, optimizer)

Epoch 1, loss: 0.107, acc: 0.828, val_loss: 0.092, val_acc: 0.842
Epoch 2, loss: 0.088, acc: 0.852, val_loss: 0.094, val_acc: 0.838
Epoch 3, loss: 0.085, acc: 0.856, val_loss: 0.098, val_acc: 0.840
Epoch 4, loss: 0.084, acc: 0.860, val_loss: 0.090, val_acc: 0.850
Epoch 5, loss: 0.082, acc: 0.861, val_loss: 0.100, val_acc: 0.839
Epoch 6, loss: 0.081, acc: 0.862, val_loss: 0.090, val_acc: 0.844
Epoch 7, loss: 0.080, acc: 0.862, val_loss: 0.090, val_acc: 0.849
Epoch 8, loss: 0.078, acc: 0.866, val_loss: 0.094, val_acc: 0.844
Epoch 9, loss: 0.078, acc: 0.867, val_loss: 0.090, val_acc: 0.849
Epoch 10, loss: 0.078, acc: 0.866, val_loss: 0.091, val_acc: 0.850


This can be further improved with longer training time and more careful hyperparameter tuning.

In [76]:
# Serialize
student_model.save_weights("student_model.weights.h5")

In [77]:
# Investigate the sizes
!ls -lh *.h5

-rw-r--r-- 1 root root 166K Jan  7 06:23 student_model.weights.h5
-rw-r--r-- 1 root root 975K Jan  7 06:22 teacher_model.weights.h5


Let's check the total number of trainable params.

In [78]:
teacher_model.summary()

Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_14 (Conv2D)              │ (None, 24, 24, 16)     │           416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_14 (MaxPooling2D) │ (None, 12, 12, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_15 (Conv2D)              │ (None, 8, 8, 32)       │        12,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_15 (MaxPooling2D) │ (None, 4, 4, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 4, 4, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_9 (Flatten)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 128)            │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 240,608 (939.88 KB)

 Trainable params: 80,202 (313.29 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 160,406 (626.59 KB)

In [79]:
student_model.summary()

Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten_10 (Flatten)            │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_20 (Dense)                │ (None, 48)             │        37,680 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_21 (Dense)                │ (None, 10)             │           490 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 38,170 (149.10 KB)

 Trainable params: 38,170 (149.10 KB)

 Non-trainable params: 0 (0.00 B)

Further size decrease is possible with TFLite.

In [80]:
# Credits: https://www.tensorflow.org/lite/performance/post_training_quant

def representative_data_gen():
    for input_value in tf.data.Dataset.from_tensor_slices(X_train).batch(1).take(100):
        yield [input_value]

def convert_to_tflite(model, tflite_file):
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    converter.representative_dataset = representative_data_gen
    converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
    converter.inference_input_type = tf.int8
    converter.inference_output_type = tf.int8
    tflite_quant_model = converter.convert()

    open(tflite_file, 'wb').write(tflite_quant_model)

In [81]:
convert_to_tflite(teacher_model, "teacher.tflite")
convert_to_tflite(student_model, "student.tflite")

Saved artifact at '/tmp/tmpt3xkad25'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32, name='keras_tensor_186')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  137555976550544: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137556445501904: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137556445496144: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137555976550160: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137556445497488: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137555976552272: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137556445501136: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137556445496528: TensorSpec(shape=(), dtype=tf.resource, name=None)


/usr/local/lib/python3.12/dist-packages/tensorflow/lite/python/convert.py:854: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


Saved artifact at '/tmp/tmpahatjll9'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32, name='keras_tensor_223')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  137556440874064: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137556440870608: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137555956598160: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137555956597200: TensorSpec(shape=(), dtype=tf.resource, name=None)


/usr/local/lib/python3.12/dist-packages/tensorflow/lite/python/convert.py:854: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


In [82]:
!ls -lh *.tflite

-rw-r--r-- 1 root root 41K Jan  7 06:23 student.tflite
-rw-r--r-- 1 root root 87K Jan  7 06:23 teacher.tflite
